In [78]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
display(HTML("<style>.container {width:100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
    
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc
import time

------------
# ==== Start ====

## Prepare username + password

In [4]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = bnpm.server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [5]:
## initialize ssh_transfer
ssh_t = bnpm.server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=bnpm.server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [6]:
## initialize ssh_compute
ssh_c = bnpm.server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=bnpm.server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [7]:
sftp = bnpm.server.sftp_interface(ssh_client=ssh_t.client)

In [8]:
prompt_snip = f'[{username}'

send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

## Find files and directories with data

In [66]:
paths_found = sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N',
    search_pattern_re='cam4.*avi',
    max_depth=5,
    find_folders=False,
    find_files=True,
    verbose=True,
)

/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231029/cam42023-10-29T10_23_54.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231024/cam42023-10-24T13_56_05.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231025/cam42023-10-25T12_51_29.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231106/cam42023-11-06T13_49_29.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231023/cam42023-10-23T11_36_48.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231022/cam42023-10-22T12_05_56.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231031/cam42023-10-31T11_52_07.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0916N/camera_data/20231027/cam4

In [67]:
import natsort

In [68]:
dates_toUse = natsort.natsorted([Path(p).parts[-2] for p in paths_found])
dates_toUse

['20231022',
 '20231023',
 '20231024',
 '20231025',
 '20231026',
 '20231027',
 '20231028',
 '20231029',
 '20231030',
 '20231031',
 '20231101',
 '20231102',
 '20231103',
 '20231104',
 '20231105',
 '20231106']

In [45]:
# dates_toUse = [
#     '20231022',
#     '20231023',
#     '20231024',
#     '20231025',
#     '20231026',
#     '20231027',
#     '20231028',
#     '20231029',
#     '20231030',
#     '20231031',
#     '20231101',
#     '20231102',
#     '20231103',
#     '20231104',
#     '20231105',
#     '20231106'
# ]

## Start an interactive job for dispatching

In [46]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=2G /bin/bash');

slurmstepd: error: *** STEP 31164719.0 ON compute-e-16-235 CANCELLED AT 2024-01-31T13:19:59 DUE TO TIME LIMIT ***
srun: Job step aborted: Waiting up to 62 seconds for job step to finish.
srun: error: compute-e-16-235: task 0: Terminated
(base) [rh183@login01 ~]$ 


## Pull/update repo

In [47]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect('git pull');

bash --pty -p interactive -t 0-00:30:00 --mem=2G /bin/ 
srun: job 31166725 queued and waiting for resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm

srun: job 31166725 has been allocated resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm

(base) [rh183@compute-a-16-162 ~]$ cd /n/data1/h
_repos/face-rhythmni/rich/github 
(base) [rh183@compute-a-16-162 face-rhythm]$ 


## Activate environment

In [48]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

git pull

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Already up-to-date.

o/sabatini/rich/virtual_envs/FRface-rhythm]$ conda activate /n/data1/hms/neurobi 



## Install face-rhythm

In [ ]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect(f'pip install -e .[all_cv2Headless] --upgrade')

In [50]:
name_env = f'/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

# names_st = ['rh183']*0 + ['joz608']*0 + ['km335']*0 + ['gyh930']*0 + ['ccb18']*0 + ['pac4988']*0 + ['aeg38']*0 + ['shl143']*1
# for date, name_st in zip(dates_toUse, names_st):
for date in dates_toUse[:]:
    cage = 'cage_0916N'
    mouse = 'mouse_0916N'

    name_slurm = mouse + '_' + date[-4:]
    
    dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/') / mouse / 'camera_data' / date)

    dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20240131' / 'ROIs_prealigned' / f'{date}' / 'ROIs.h5')

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20240131' / date)


    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server



    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
        'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_videos} \
    {dir_ROIs} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    ## Upload `dispatcher.py` file
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));


    send_expect(commands['dispatch']);


    ## For serverteeny usage
#     send_expect(f"chmod 777 {path_dispatcher_remote}")
#     run_file = {
#         "name": "Rich",
#         "o2_acct": str(name_st),
#         "notes": f"basic_run_{mouse}_{date}",
#         "command": f"sbatch /n/data1/hms/neurobio/sabatini/rich/serverteeny_submitter.sh {commands['dispatch']}",
#     }
#     path_temp = str(Path(tempfile.gettempdir()) / f'run_{date}.json')
#     bnpm.file_helpers.json_save(
#         obj=run_file,
#         filepath=path_temp,
#     )
#     sftp.sftp.put(
#         localpath=path_temp,
#         remotepath=str(Path('/n/data1/hms/neurobio/sabatini/serverteeny/run') / Path(path_temp).name),
#     );
        

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230702/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230702     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230702     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230702/ROIs.h5     jobNum_     mouse_0403R_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR
Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709/run_pipeline_basic.py
chmod: cannot access ‘/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230709/ROIs.h5’: No such file or directory
Submitting job: jobNum_ 0
Submitted batch job 31166849
ce-rhythm]$ 


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230703     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230703     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230703/ROIs.h5     jobNum_     mouse_0403R_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

ouse_0403R_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FRum_     m 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230703     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230703     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230703/ROIs.h5     jobNum_     mouse_0403R_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230702/run_pipeline_basic.py

Submi

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230706     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230706/ROIs.h5     jobNum_     mouse_0403R_0706     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

ouse_0403R_0706     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FRum_     m 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230707/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230707     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230707     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230707/ROIs.h5     jobNum_     mouse_0403R_0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706/run_pipeline_basic.py

Submi

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230708     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230708/ROIs.h5     jobNum_     mouse_0403R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

ouse_0403R_0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FRum_     m 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230708     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230708/ROIs.h5     jobNum_     mouse_0403R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230707/run_pipeline_basic.py
Submit

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230709     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230709/ROIs.h5     jobNum_     mouse_0403R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

ouse_0403R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FRum_     m 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230709     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/ROIs_prealigned/20230709/ROIs.h5     jobNum_     mouse_0403R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/run_pipeline_basic.py

Submi

In [34]:
# ssh_c.expect()

bash
srun: job 31164719 queued and waiting for resources
srun: job 31164719 has been allocated resources
(base) [rh183@compute-e-16-235 ~]$ 


('srun --pty -p interactive -t 0-00:30:00 --mem=2G /bin/ \rbash\r\nsrun: job 31164719 queued and waiting for resources\r\nsrun: job 31164719 has been allocated resources\r\n(base) [rh183@compute-e-16-235 ~]$ ',
 True)

## Prepare directories

In [59]:
import numpy as np

mouse = 'mouse_00916N'
date = '20230420'

name_slurm = date[-4:]

dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments') / mouse / 'camera_data' / date)

dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')

dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date)


name_job = 'jobNum_'


path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server


## Prepare commands

In [60]:
## Expectation for dispatch args: 
# path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env

commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
#     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
python3 \
{path_dispatcher_remote} \
{path_script_remote} \
{dir_saveOutputs} \
{dir_videos} \
{dir_ROIs} \
{name_job} \
{name_slurm} \
{name_env}",
}
display(commands)

{'dispatch': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230420/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/20230420 /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322R/camera_data/20230420 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322R/run_20230701/ROIs_prealigned/mouse_0322R__20230420/ROIs.h5 jobNum_ 0420 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

## Upload `dispatcher.py` file

In [61]:
sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch

In [62]:
send_expect(commands['dispatch']);

_repos/face-rhythmurobio/sabatini/rich/github 
(base) [rh183@compute-e-16-191 face-rhythm]$ git pull



In [19]:
# for date in dates_toUse:
for date in dates_toUse[:]:
    # mouse = 'mouse_0403L'
    mouse = 'mouse_0322N'
    # date = '20230420'
    
    name_slurm = date[-4:]
    
    dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments') / mouse / 'camera_data' / date)
    
    # dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230903' / 'ROIs_prealigned' / f'{date}' / 'ROIs.h5')
    dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')
    
    # dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230903' / date)
    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date)
    
    
    
    name_job = 'jobNum_'
    
    
    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/run_pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server
    
    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server
    
    
    ## Prepare commands
    
    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env
    
    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
        'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_videos} \
    {dir_ROIs} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    ## Upload `dispatcher.py` file
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    ## Dispatch
    
    # send_expect(commands['dispatch']);

    # time.sleep(60)

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230430     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230430/ROIs.h5     jobNum_     0430     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230501/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230501     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230501     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230501/ROIs.h5     jobNum_     0501     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230502     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230502/ROIs.h5     jobNum_     0502     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230503     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230503/ROIs.h5     jobNum_     0503     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230504/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230504     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230504     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230504/ROIs.h5     jobNum_     0504     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230528     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230528/ROIs.h5     jobNum_     0528     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230529     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230529/ROIs.h5     jobNum_     0529     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230530     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230530/ROIs.h5     jobNum_     0530     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230531     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230531/ROIs.h5     jobNum_     0531     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0322N/camera_data/20230601     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230601/ROIs.h5     jobNum_     0601     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

In [20]:
paths_all = sftp.search_recursive(
    path=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/',
    search_pattern_re='VQT_Analyzer.h5',
    max_depth=8,
)

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230512/

In [21]:
{p: f['last_modified'] for p, f in sftp.get_fileProperties(paths_all, error_on_missing=True).items()}

{'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 16:19:36',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230501/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 16:17:37',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 19:43:05',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 16:17:38',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230504/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 16:19:36',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/jobNum_0/analysis_files/VQT_Analyzer.h5': '2023-07-03 16:19:36',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/

In [ ]:
send_expect('pip uninstall face-rhythm')

In [58]:
send_expect('y')

y

  Successfully uninstalled face-rhythm-0.1.3

ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac


('(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac\rce-rhythm]$ ',
 True)

In [60]:
send_expect('pip install -e .')

pip 
install -e .

Obtaining file:///n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm

  Preparing metadata (setup.py) ... -
done


  Running setup.py develop for face-rhythm


ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac


('\r\n(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac\rce-rhythm]$ ',
 True)

In [61]:
send_expect('pip list')

pip 
list

Package                        Version    Editable project location
------------------------------ ---------- ------------------------------------------------------------
anyio                          3.7.0
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
arrow                          1.2.3
asttokens                      2.2.1
async-lru                      2.0.2
attrs                          23.1.0
Babel                          2.12.1
backcall                       0.2.0
bcrypt                         4.0.1
beautifulsoup4                 4.12.2
bleach                         6.0.0
blosc2                         2.0.0
bnpm                           0.2.4
ccimport                       0.3.7
certifi                        2023.5.7
cffi                           1.15.1
charset-normalizer             3.1.0
cmake                          3.26.4
comm                           0.1.3
configparser                   5.3.0
contourpy                      

('(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac\rce-rhythm]$ ',
 True)

In [62]:
send_expect()

ls

batch_run                 face_rhythm           notebooks         setup.py
build                     face_rhythm.egg-info  old               test_data
docs                      LICENSE               README.md         tox.ini
environment_CPU_only.yml  lucas_kanade.py       requirements.txt
environment_GPU.yml       Makefile              scripts
ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac


('\x1b\x1bbatch_run\x1b                 \x1bface_rhythm\x1b           \x1bnotebooks\x1b         setup.py\r\n\x1bbuild\x1b                     \x1bface_rhythm.egg-info\x1b  \x1bold\x1b               \x1btest_data\x1b\r\n\x1bdocs\x1b                      LICENSE               README.md         tox.ini\r\nenvironment_CPU_only.yml  lucas_kanade.py       requirements.txt\r\nenvironment_GPU.yml       Makefile              \x1bscripts\x1b\r\n(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-229 fac\rce-rhythm]$ ',
 True)

## Run with serverteeny

### check on job

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()